In [1]:
!pip install prettytable

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from prettytable import PrettyTable

# Load dataset (assuming it's already loaded as 'df')
df = pd.read_csv(r"C:\devopspro2\app\restaurant_data.csv")



# Select features for clustering
features = ['Rating', 'Average Cost for Two (INR)', 'Total Reviews']
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[features])

# Fit K-Means
kmeans = KMeans(n_clusters=3, random_state=42)  # Adjust n_clusters as needed
df['Cluster'] = kmeans.fit_predict(scaled_features)

# Function to recommend restaurants based on city, rating, and price constraints
def recommend_restaurants_by_city_and_constraints(user_city, min_rating, max_price, user_cluster=None):
    user_city = user_city.lower()

    # Check if the entered city exists in the dataset
    available_cities = df['City'].str.lower().unique()
    if user_city not in available_cities:
        print(f"Sorry, we couldn't find any restaurants in the city '{user_city}'.")
        return pd.DataFrame()

    # Filter restaurants in the specified city
    city_restaurants = df[df['City'].str.lower() == user_city]

    # Apply cluster filtering if user_cluster is specified
    if user_cluster is not None:
        city_restaurants = city_restaurants[city_restaurants['Cluster'] == user_cluster]

    # Apply rating and price constraints
    filtered_restaurants = city_restaurants[
        (city_restaurants['Rating'] >= min_rating) &
        (city_restaurants['Average Cost for Two (INR)'] <= max_price)
    ]

    return filtered_restaurants

# Get user input for city, rating, and price
user_city = input("Enter the city: ").strip()
min_rating = float(input("Enter minimum rating (e.g., 4.0): "))
max_price = float(input("Enter maximum price for two (e.g., 1000): "))

# Recommend restaurants based on city, rating, and price constraints
recommended_restaurants = recommend_restaurants_by_city_and_constraints(user_city, min_rating, max_price)

# Display the recommended restaurants using PrettyTable if there are results
if recommended_restaurants.empty:
    print("No restaurants found with the specified criteria.")
else:
    # Create a PrettyTable object to display the output in a tabular format
    table = PrettyTable()
    table.field_names = ["Location", "Name", "City", "Cuisine", "Rating", "Total Reviews", "Average Cost for Two (INR)"]

    # Add rows to the table
    for _, row in recommended_restaurants.iterrows():
        table.add_row([row['Location'], row['Name'], row['City'], row['Cuisine'], row['Rating'], row['Total Reviews'], row['Average Cost for Two (INR)']])

    # Print the table
    print(f"\nRecommended Restaurants in {user_city.capitalize()} with rating >= {min_rating} and price <= {max_price}:")
    print(table)


Recommended Restaurants in Delhi with rating >= 4.3 and price <= 500.0:
+----------+--------------+-------+---------+--------+---------------+----------------------------+
| Location |     Name     |  City | Cuisine | Rating | Total Reviews | Average Cost for Two (INR) |
+----------+--------------+-------+---------+--------+---------------+----------------------------+
| MG Road  | Pizza Palace | Delhi |   Thai  |  4.8   |      2586     |            343             |
+----------+--------------+-------+---------+--------+---------------+----------------------------+
